In [2]:
import sqlite3
import pandas as pd
import zipfile
import io
import numpy as np

In [3]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import gensim
from gensim.models import Word2Vec

In [4]:
#! pip install -U sentence-transformers

In [5]:
conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [6]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [7]:
df = df[:10]

# Semantic search engine

In [8]:
def func1(a):
    with io.BytesIO(a) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    srt_content = subtitle_content.decode('latin-1').split('\r\n')
    temp = ''
    for i in srt_content:
        if '-->' not in i and i != '' and not i.isdigit():
            temp+=i
    temp = temp.replace('</i>', '').replace('<i>', '').replace("\\" ,  "")
    return temp

In [9]:
df.content = df.content.apply(func1)

# data pre-processing along with chunking

In [10]:
def pre_processing(raw_text):
    #removing special characters
    temp = re.sub('[^a-zA-Z]',' ',raw_text)

    temp = temp.lower()
    
    #tokenizing 
    tokens = temp.split()
    
    #removing stop words
    tokens1 = [i for i in tokens if i not in stopwords.words('english')]
    
    #steming 
    stem = PorterStemmer()
    tokens2 = [stem.stem(i) for i in tokens1]
               
    
    t_range = [i for i in range(0,int(len(tokens2)),768) ]

    if len(t_range) == 1:
        return ' '.join(tokens2)
    else :
        ch_array = []

        for i in range(int(len(t_range)-1)):
            ch_array.append(tokens2[t_range[i]:t_range[i+1]])

        ch_array1 = [] 

        for i in ch_array:
            ch_array1.append(' '.join(i))

        return ch_array1

In [11]:
df.content = df.content.apply(pre_processing)

In [12]:
df = df.explode('content').reset_index(drop = True)

In [13]:
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,watch video onlin open subtitlesfre browser ex...
1,9180533,the.message.(1976).eng.1cd,told mother afterward minut tender littl grip ...
2,9180533,the.message.(1976).eng.1cd,year worshipedwood stone imag manufactur live ...
3,9180533,the.message.(1976).eng.1cd,choosebetween mani welcom god guid camel stop ...
4,9180533,the.message.(1976).eng.1cd,soon enough big armi three thousand men hundr ...


In [14]:
from sentence_transformers import SentenceTransformer, util

In [15]:
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

In [16]:
df['embeddings'] = df.content.apply(lambda a : model.encode(a))

In [17]:
df.head()

,num,name,content,embeddings
0,9180533,the.message.(1976).eng.1cd,watch video onlin open subtitlesfre browser ex...,"[-0.18982282, 0.1759449, -0.3058828, 0.0562893..."
1,9180533,the.message.(1976).eng.1cd,told mother afterward minut tender littl grip ...,"[-0.09383225, 0.15431398, -0.35503858, -0.0373..."
2,9180533,the.message.(1976).eng.1cd,year worshipedwood stone imag manufactur live ...,"[-0.06360224, 0.3060157, -0.33637393, 0.158720..."
3,9180533,the.message.(1976).eng.1cd,choosebetween mani welcom god guid camel stop ...,"[-0.1425536, 0.07811344, -0.3047243, -0.061560..."
4,9180533,the.message.(1976).eng.1cd,soon enough big armi three thousand men hundr ...,"[-0.18743579, 0.08985154, -0.3369387, -0.02587..."


In [19]:
query = 'This is a query'

# Calculating Cosine Similarity

In [60]:
df['c_sim'] = df.embeddings.apply(lambda a : np.array(util.cos_sim(model.encode(pre_processing(query)),a))[0][0])

In [78]:
movie = ''
for i in df[df.c_sim == df.c_sim.agg('max')]['name']:
    movie+= i
movie = movie.replace('.',' ')

'rudrabinar obhishaap s02 e01 swaralipir kut taan (2022) eng 1cd'

AttributeError: 'Series' object has no attribute 'split'

In [ ]:
df.c_sim

In [ ]:
query = "didn't heard about you"

In [ ]:
def c_sim(a):
    

In [ ]:
len(df.content[0].split())

In [ ]:
def chunk(a):
    for i in a

In [ ]:
df.content[0][:1]